# ETL Project 

---
This project's purpose is to work with a partner to find multiple datasets or data sources and load them together for future analysis. We (Sarah & Paul) are interested in health patterns across the US, so this interest guided our search. Notes are included as we go.


In [37]:
%matplotlib inline

# import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Import image from web scraping

In [30]:
#scraping a web page to bring in an image to welcome you to our project
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#found the image on google and it linked to this medium article
url = 'https://medium.ip.sx/meet-our-new-project-manager-ovidiu-rauca-263e4d6a6cfc'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [34]:
results = soup.find('img',class_='progressiveMedia-image js progressiveMedia-image')
img_url = results.find('src')

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(img_url)

## Load Mortality Data
We found data on Mortality Rates in the United States from CDC.gov: https://wonder.cdc.gov/controller/datarequest/D140
We decided to export this dataset based on:
* County code
* ICD Chapter (type of death)
* death count
* population of county

The data exported as a txt file. Below I'm reading in the file to pandas like it is a csv, using "\t" to indicate the tab delimited separations between the data cells.

In [5]:
data = pd.read_csv('Deaths.txt', sep="\t",)
data.head()

,Notes,County,County Code,ICD Chapter,ICD Chapter Code,Deaths,Population,Crude Rate,% of Total Deaths
0,NaN,"Autauga County, AL",1001.0,Certain infectious and parasitic diseases,A00-B99,211.0,918492.0,23.0,0.0%
1,NaN,"Autauga County, AL",1001.0,Neoplasms,C00-D48,1771.0,918492.0,192.8,0.0%
2,NaN,"Autauga County, AL",1001.0,Diseases of the blood and blood-forming organs...,D50-D89,44.0,918492.0,4.8,0.0%
3,NaN,"Autauga County, AL",1001.0,"Endocrine, nutritional and metabolic diseases",E00-E88,354.0,918492.0,38.5,0.0%
4,NaN,"Autauga County, AL",1001.0,Mental and behavioural disorders,F01-F99,218.0,918492.0,23.7,0.0%


In [18]:
# drop notes column

# data.drop(["Notes"], inplace="True")
data.head()

,County,County Code,ICD Chapter,ICD Chapter Code,Deaths,Population,Crude Rate,% of Total Deaths
0,"Autauga County, AL",1001.0,Certain infectious and parasitic diseases,A00-B99,211.0,918492.0,23.0,0.0%
1,"Autauga County, AL",1001.0,Neoplasms,C00-D48,1771.0,918492.0,192.8,0.0%
2,"Autauga County, AL",1001.0,Diseases of the blood and blood-forming organs...,D50-D89,44.0,918492.0,4.8,0.0%
3,"Autauga County, AL",1001.0,"Endocrine, nutritional and metabolic diseases",E00-E88,354.0,918492.0,38.5,0.0%
4,"Autauga County, AL",1001.0,Mental and behavioural disorders,F01-F99,218.0,918492.0,23.7,0.0%


## Load FIPS Data

In [36]:
fips_df = pd.read_csv('fipscode_breakout.csv')
fips_df.head()

,state,state_fips,county_fips,county,class_code
0,AL,1,1,Autauga County,H1
1,AL,1,3,Baldwin County,H1
2,AL,1,5,Barbour County,H1
3,AL,1,7,Bibb County,H1
4,AL,1,9,Blount County,H1


## Load into SQLite
Put data manually into SQLITE APP, then read it into this notebook for viewing and mapping.

In [38]:
# set up SQLite DB
engine = create_engine("sqlite:///Resources/USAhealth.sqlite")

In [39]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

OperationalError: (sqlite3.OperationalError) unable to open database file (Background on this error at: http://sqlalche.me/e/e3q8)